In [1]:
import os
# https://discuss.pytorch.org/t/how-to-change-the-default-device-of-gpu-device-ids-0/1041/24
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=f"{1}"

import torch
print(f"current_device:{torch.cuda.current_device()}")
print(f"device_count:{torch.cuda.device_count()}")
device = torch.device("cuda")


import sys
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from ddpg import DDPGagent
from ppo.PPO_continuous import PPO,Memory
# from ddpg.utils import NormalizedEnv
from flexipod_env import FlexipodEnv

env = FlexipodEnv(dof = 12)
# env = NormalizedEnv(env)

############## Hyperparameters ##############
# env_name = "BipedalWalker-v3"
env_name = "flexipod"
render = True
solved_reward = 1500        # stop training if avg_reward > solved_reward
log_interval = 80           # print avg reward in the interval
# log_interval = 2           # print avg reward in the interval

max_episodes = 20000        # max training episodes
max_timesteps = 1500        # max timesteps in one episode

# update_timestep = 4000      # update policy every n timesteps
update_timestep = 3000      # update policy every n timesteps


# action_std = 1.0            # constant std for action distribution (Multivariate Normal)
action_std = 0.8            # constant std for action distribution (Multivariate Normal)
K_epochs = 80               # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr = 0.0002                 # parameters for Adam optimizer
betas = (0.9, 0.999)

random_seed = None
#############################################
# creating environment
# env = gym.make(env_name)
state_dim = np.prod(env.observation_space.shape)
action_dim = env.action_space.shape[0]

current_device:0
device_count:1


In [2]:
from torch.utils.tensorboard import SummaryWriter
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/soft12dof_experiment_tmp')

In [77]:
# import time
# env.reset()
# for k in range(100):
#     env.step(np.ones(12))
#     time.sleep(0.01)

[ 1.5708227e+00 -1.5707978e+00  1.6463336e+00 -1.6463970e+00
  1.5709275e+00 -1.7923447e-06 -1.5709660e+00  3.2216233e-06
  9.5009623e-04  7.5442471e-02 -7.8507821e-04  7.5444609e-02]


array([[-2.63889651e-05,  1.00000000e+00, -1.47422281e-06,
        -1.00000000e+00, -7.54654333e-02,  9.97148395e-01,
        -7.55286738e-02, -9.97143626e-01, -1.31173932e-04,
         1.00000000e+00,  1.00000000e+00, -1.79234473e-06,
        -1.69678533e-04, -1.00000000e+00,  1.00000000e+00,
         3.22162327e-06,  9.99999523e-01,  9.50096059e-04,
         9.97155547e-01,  7.53709227e-02,  9.99999702e-01,
        -7.85078155e-04,  9.97155428e-01,  7.53730610e-02,
         8.22463189e-04, -5.88030671e-04,  6.82034204e-03,
        -7.78745394e-03,  4.89542261e-03, -6.77526114e-05,
        -5.65027399e-03,  1.32275512e-04,  1.19642196e-02,
         1.13112004e-02, -7.85613526e-03,  1.10748718e-02,
        -1.61225660e-04,  7.36658985e-05, -1.14613236e-03,
         1.41272275e-03, -8.97482270e-04,  1.23662803e-05,
         1.07941614e-03, -2.34802847e-05, -3.69631103e-03,
        -1.45579700e-03,  2.81199324e-03, -1.43269624e-03,
        -2.67469004e-04,  7.23372125e-07,  9.99999940e-0

In [3]:
# env = FlexipodEnv(dof = 12)
# self = env
# msg_rec,_,_,_ = env.step()

In [ ]:
if random_seed:
    print("Random Seed: {}".format(random_seed))
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip)
print(lr,betas)

# logging variables
running_reward = 0
avg_length = 0
max_avg_length = 0
time_step = 0

# checkpoint = ppo.load(f'./PPO_continuous_{env_name}_best.pth')
# checkpoint = ppo.load(f'./PPO_continuous_{env_name}.pth')
# max_avg_length = checkpoint["avg_length"]

# training loop
for i_episode in range(0, max_episodes+1):
    state = env.reset()
    for t in range(max_timesteps):
        time_step +=1
        # Running policy_old:
        action = ppo.select_action(state, memory)
        state, reward, done, _ = env.step(action)

        # Saving reward and is_terminals:
        memory.rewards.append(reward)
        memory.is_terminals.append(done)

        # update if its time
        if time_step % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            time_step = 0
        running_reward += reward
        if render:
            env.render()
        if done:
            break

    avg_length += t

    # save every 500 episodes
    if i_episode % 500 == 0:
        ppo.save(f'./PPO_continuous_{env_name}.pth',avg_length=avg_length)

    # logging
    if i_episode % log_interval == 0:
        avg_length = avg_length/log_interval
        running_reward = running_reward/log_interval
        writer.add_scalar("avg_length/train", avg_length, i_episode)
        writer.add_scalar("running_reward/train", running_reward, i_episode)
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            ppo.save(f'./PPO_continuous_solved_{env_name}.pth',avg_length=avg_length)
            break
            
        if avg_length>max_avg_length:
            max_avg_length = avg_length
            ppo.save(f'./PPO_continuous_{env_name}_best.pth',avg_length=avg_length)
        elif np.random.random()<0.1:# 50% chance 
            checkpoint = ppo.load(f'./PPO_continuous_{env_name}_best.pth')
            print(f"load old best,avg_length={checkpoint['avg_length']}")# restart

        print(f'Episode {i_episode} \t Avg length: {avg_length:.0f} \t Avg reward: {running_reward:.0f}')
        running_reward = 0
        avg_length = 0
        
env.pause()

0.0002 (0.9, 0.999)
Episode 0 	 Avg length: 1 	 Avg reward: 1


In [13]:
# writer.add_scalar("baseline_length/train", 200, 0)
# writer.add_scalar("baseline_length/train", 200, log_interval)

In [2]:
memory_2 = Memory()

max_timesteps=3000
action_std=.1
ppo = PPO(state_dim, action_dim, action_std, lr, betas, gamma, K_epochs, eps_clip)
ppo.load(f'./PPO_continuous_{env_name}_best.pth')

time_steps = []
durations = []
for k in range(8):
    state = env.reset()
    for t in range(max_timesteps):
        # Running policy_old:
        action = ppo.select_action(state, memory_2)
        state, reward, done, info = env.step(action)
#         state, reward, done, info = env.step()
        if done or t==max_timesteps-1:
            time_steps.append(t)
            episode_duration= info['t'] - env.episode_start_time
            durations.append(episode_duration)
            print(f"time steps:{t} ,duration: {episode_duration:.2f}[s], fps:{t/episode_duration:.0f}")
            break
            
print(f"mean time steps:{np.mean(time_steps):.0f} ,duration: {np.mean(durations):.2f}[s]")

time steps:270 ,duration: 3.64[s], fps:74
time steps:282 ,duration: 3.30[s], fps:85
time steps:291 ,duration: 3.32[s], fps:88
time steps:1131 ,duration: 12.79[s], fps:88
time steps:430 ,duration: 5.15[s], fps:83


D:\repo\FlexipodFast\src\python\flexipod_env.py:126: UserWarning: step(): try #0:timed out
  warnings.warn(f"step(): try #{k}:{e}")
D:\repo\FlexipodFast\src\python\flexipod_env.py:126: UserWarning: step(): try #1:timed out
  warnings.warn(f"step(): try #{k}:{e}")


time steps:1107 ,duration: 12.96[s], fps:85
time steps:336 ,duration: 4.01[s], fps:84
time steps:497 ,duration: 5.81[s], fps:85
mean time steps:543 ,duration: 6.37[s]


In [3]:
%load_ext line_profiler
state = env.reset()
memory_2 = Memory()
def test():
    for k in range(100):
        ppo.select_action(state, memory_2)
    
%lprun -f ppo.policy_old.act test()
# %lprun -f ppo.select_action test()



UsageError: Could not find function 'ppo.policy_old.act'.
NameError: name 'ppo' is not defined


In [3]:
torch.__version__

'1.7.0'